# new RNN

In [52]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [53]:
# ***************************************************
# RNN Model 
# ***************************************************

class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_voxels):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        
        # For each time step, predict a vector of beta values for each voxel
        self.fc = nn.Linear(hidden_size, num_voxels)  # Output: one prediction per voxel at each time step

    def forward(self, x):
        # x: (batch_size, time_steps, num_voxels)
        batch_size, time_steps, num_voxels = x.size()  # Correctly unpack the shape
        
        # Pass through RNN (output: batch_size, time_steps, hidden_size)
        rnn_out, _ = self.rnn(x)
        
        # Apply fully connected layer to predict beta values for each voxel at each time step
        # output: (batch_size, time_steps, num_voxels)
        output = self.fc(rnn_out)
        
        return output  # Shape: (batch_size, time_steps, num_voxels)

        

In [54]:

# ***************************************************
# Training Function
# ***************************************************

def train(model, criterion, dataset_train, dataset_test, optimizer, num_epochs, device):
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        total_loss = 0
        
        # Iterate through the training data
        for batch_x, batch_y in dataset_train:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            
            # Reshaping batch_x to (batch_size, time_steps, 1)
            batch_x = batch_x.unsqueeze(-1)  # Adding extra dimension for input size
            # Ensure the target labels have the correct shape (expand to match (batch_size, time_steps, num_voxels))
            batch_y = batch_y.expand(-1, -1, num_voxels)  # Expand to match the output shape (32, 100, 200)
            
            # Forward pass through the model
            predictions = model(batch_x)  # Shape: (batch_size, time_steps, num_voxels)

            # Compute loss (Mean Squared Error for regression task)
            loss = criterion(predictions, batch_y)  # batch_y: (batch_size, time_steps, num_voxels)
            total_loss += loss.item()  # Accumulate the loss over the batch

            # Backpropagation
            optimizer.zero_grad()  # Zero the gradients before the backward pass
            loss.backward()  # Backpropagate the loss
            optimizer.step()  # Update the model parameters

        # Average loss over the entire training dataset
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(dataset_train)}")

        # Test the model
        model.eval()  # Set model to evaluation mode (disables dropout, batchnorm, etc.)
        total_test_loss = 0

        # Iterate through the testing data
        with torch.no_grad():  # No gradients are needed for evaluation
            for batch_x, batch_y in dataset_test:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                batch_x = batch_x.unsqueeze(-1)  # Reshape for evaluation

                predictions = model(batch_x)

                # Compute the test loss
                loss = criterion(predictions, batch_y)
                total_test_loss += loss.item()

        # Average test loss over the entire test dataset
        print(f"Epoch {epoch+1}/{num_epochs}, Test Loss: {total_test_loss / len(dataset_test)}")


In [55]:
# ***************************************************
# Accuracy Calculation
# ***************************************************

def accuracy(predicted_logits, reference):
    """
    Compute the ratio of correctly predicted labels.
    """
    # Print the shape of predicted logits for debugging
    # print(f"Predicted logits shape: {predicted_logits.shape}")
    # print(f"Reference shape: {reference.shape}")

    # predicted_logits is expected to be of shape [batch_size * time_steps, num_classes]
    # reference is expected to be of shape [batch_size * time_steps]
    
    # Get predicted class for each time step (the class with the highest logit)
    predicted_classes = predicted_logits.argmax(dim=1)  # Shape: [batch_size * time_steps]
    
    # Count how many predictions match the reference labels
    correct_predictions = (predicted_classes == reference).sum()  # Number of correct predictions
    accuracy = correct_predictions.float() / reference.numel()  # Compute accuracy
    
    return accuracy



In [56]:

# ***************************************************
# Dataset Preparation
# ***************************************************

# Generate synthetic data (for example, num_voxels samples)
def generate_synthetic_data(num_voxels, time_steps):
    # X and Y: Shape (num_voxels, time_steps), i.e., each voxel is a sample
    X = torch.randn(num_voxels, time_steps)  # Time series data for each voxel (num_voxels samples)
    Y = torch.randn(num_voxels, time_steps)  # Beta values for each voxel at each time step
    
    # Reshaping Y to match (batch_size, time_steps, num_voxels)
    Y = Y.unsqueeze(2)  # Add the voxel dimension
    
    return X, Y

In [57]:
# Hyperparameters
num_epochs = 100
learning_rate = 1e-3
batch_size = 32
time_steps = 100
num_voxels = 200  # Number of voxels (samples)
hidden_size = 64  # Hidden size of the RNN
num_beta = time_steps  # one beta array per voxel


X_train, Y_train = generate_synthetic_data(num_voxels, time_steps)
X_test, Y_test = generate_synthetic_data(num_voxels, time_steps)

print(X_train.shape)

# Create datasets and data loaders
dataset_train = DataLoader(
    TensorDataset(X_train, Y_train),
    batch_size=batch_size,
    shuffle=True,
)

dataset_test = DataLoader(
    TensorDataset(X_test, Y_test),
    batch_size=batch_size,
    shuffle=False,
)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model, criterion, optimizer
model = RNNModel(input_size=1, hidden_size=hidden_size, num_voxels=num_voxels).to(device)
criterion = nn.MSELoss()  # Mean Squared Error for regression task (beta values)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Start training
train(model, criterion, dataset_train, dataset_test, optimizer, num_epochs, device)

torch.Size([200, 100])
Epoch 1/100, Loss: 1.0027454836027963
Epoch 1/100, Test Loss: 0.9857481036867414
Epoch 2/100, Loss: 0.9866312657083783
Epoch 2/100, Test Loss: 0.9833007454872131
Epoch 3/100, Loss: 0.9852571402277265
Epoch 3/100, Test Loss: 0.982191128390176
Epoch 4/100, Loss: 0.9951179964201791
Epoch 4/100, Test Loss: 0.9818691526140485
Epoch 5/100, Loss: 0.9915696552821568
Epoch 5/100, Test Loss: 0.9817608169146946
Epoch 6/100, Loss: 0.9815789546285357
Epoch 6/100, Test Loss: 0.9817618812833514
Epoch 7/100, Loss: 0.9945144397871835
Epoch 7/100, Test Loss: 0.9818718177931649
Epoch 8/100, Loss: 0.9813340902328491
Epoch 8/100, Test Loss: 0.9817350932529995
Epoch 9/100, Loss: 0.9975192206246513
Epoch 9/100, Test Loss: 0.9816465888704572
Epoch 10/100, Loss: 0.9892709170069013
Epoch 10/100, Test Loss: 0.9816653728485107
Epoch 11/100, Loss: 0.9884876949446542
Epoch 11/100, Test Loss: 0.9816250460488456
Epoch 12/100, Loss: 0.9846988575799125
Epoch 12/100, Test Loss: 0.9816958819116864
